# *** Clean-up of GIS data to be used in SFRmaker scripts(s). ***

As I was struggling with NBr37, I decided it'd be a good idea to first clean-up the GIS data and save them to a cleaned-up file, and only then use them in the SFRmaker scripts.

# 0. Imports

In [1]:
import os
from os import listdir as LD, makedirs as MDs
from os.path import join as PJ, basename as PBN, dirname as PDN, exists as PE
import shutil as sh
import pandas as pd
from datetime import datetime as DT
import matplotlib.pyplot as plt
from pathlib import Path
import re

In [2]:
import WS_Mdl.utils as U
import WS_Mdl.utils_imod as UIM
import WS_Mdl.calcs as C
import WS_Mdl.geo as G
from WS_Mdl.utils import bold, style_reset

In [3]:
import importlib as IL
IL.reload(U)
IL.reload(UIM)
IL.reload(G)

<module 'WS_Mdl.geo' from 'G:\\code\\WS_Mdl\\geo.py'>

In [4]:
# Import sfrmaker and other necessary packages for SFR network creation
import geopandas as gpd
import numpy as np
import xarray as xr
import shapely
from shapely.geometry import LineString, MultiLineString, box
from shapely.ops import linemerge, split, unary_union

In [5]:
from itables import init_notebook_mode
init_notebook_mode(all_interactive=True)

# 1. Options

In [6]:
Pa_Gpkg_In = U.Pa_WS + r"\models\NBr\PrP\SFR\BrabantseDelta\acceptatiedatabase.gdb"
detailed = 'hydroobject'
primary = 'LEGGER_VASTGESTELD_WATERLOOP_CATEGORIE_A'
MdlN = 'NBr37'

In [7]:
U.set_verbose(False)
# Load paths and variables from PRJ & INI
d_Pa = U.get_MdlN_Pa(MdlN)
Pa_PRJ = d_Pa['PRJ']
Dir_PRJ = PDN(Pa_PRJ)
d_INI = U.INI_to_d(d_Pa['INI'])
Xmin, Ymin, Xmax, Ymax, cellsize, N_R, N_C = U.Mdl_Dmns_from_INI(d_Pa['INI'])
SP_date_1st, SP_date_last = [DT.strftime(DT.strptime(d_INI[f'{i}'], '%Y%m%d'), '%Y-%m-%d') for i in ['SDATE', 'EDATE']]
dx = dy = float(d_INI['CELLSIZE'])
U.Mdl_Dmns_from_INI(d_Pa['INI'])

(113100.0, 387600.0, 125100.0, 396200.0, 25.0, 344, 480)

In [8]:
l_X_Y_Cols = ['Xa', 'Ya', 'Xz', 'Yz']

# 2. Create SFR lines

## 2.1.Load

In [9]:
A0 = gpd.read_file(Pa_Gpkg_In, layer=primary) # A is the primary Gpkg layer
A0_ = A0.copy()

## 2.2. Review

### 2.2.0 Create X & Y columns

In [10]:
A1 = U.Calc_GDF_XY_start_end_from_Geom(A0)

### 2.2.1 .describe()

In [11]:
A1.shape

(12864, 59)

In [12]:
A1.describe()

Loading ITables v2.5.2 from the init_notebook_mode cell... (need help?)


It's clear that a lot of the columns have very little data. So I will remove those columns as there isn't much we can do with them, but it would take time to analyze them.

### 2.2.2 Check number of values and remove columns with little data.

In [13]:
U.DF_Col_value_counts_grouped(A1, percentile_step=10)

Loading ITables v2.5.2 from the init_notebook_mode cell... (need help?)


This confirms that most columns have very few values. We'll drop all columns that have fewer valid values than 10% of the length of the DataFrame.

In [14]:
l_GDF_Cols_to_keep = [col for col in A1.columns if A1[col].notnull().sum() >= (0.1 * len(A1))]
print(f"{len(l_GDF_Cols_to_keep)}/{A1.shape[1]} columns kept in A1.")
A2 = A1[l_GDF_Cols_to_keep].copy()

24/59 columns kept in A1.


A lot of columns remain, but their number has reduced a lot. Let's check them, and decide which to keep. We'll proceed with a joint GDF called GDF.

In [15]:
A2.shape

(12864, 24)

### 2.2.3 Narrow down to SFRmaker columns.

Truth is we only need a few columns for SFRmaker to work. We'll proceed with just those columns, but the rest could potentially be reviewed and saved as a .shp/.gpkg file later on.

In [16]:
A2.describe(include='all')

Loading ITables v2.5.2 from the init_notebook_mode cell... (need help?)


1. **CODE**:                        OK           -          All unique, as we want them to be.
2. **WS_STATUS_L**:                 OK           -          All filled. Seems ok.
3. **WS_LEGGERCATEGORIE_L**:        OK           -          "   "   "
4. **WS_LEGGERBRON_L**:             OK           -          
5. **DATUM_VASTGESTELD**:           OK           -           
6. **LEGGER_KENMERK**:              OK           -           
7. **WS_BODEMBREEDTE_L**:           OK           -           
8. **WS_BH_BOVENSTROOMS_L**:        OK           -           
9. **WS_BH_BENEDENSTROOMS_L**:      OK           -           
10. **WS_TALUD_LINKS_L**:           OK           -           
11. **WS_TALUD_RECHTS_L**:          OK           -           
12. **WS_BIJZ_FUNCTIE_L**:          OK           -           
13. **WIJZIGING**:                  OK           -           
14. **CREATED_USER**:               OK           -           
15. **CREATED_DATE**:               OK           -           
16. **LAST_EDITED_USER**:           OK           -           
17. **LAST_EDITED_DATE**:           OK           -           
18. **WS_LEGGERVERWIJZING_L**:      OK           -           
19. **SHAPE_Length**:               OK           -           
20. **geometry**:                   OK           -           
21. **Xstart**:                     OK           -           
22. **Ystart**:                     OK           -           
23. **Xend**:                       OK           -           
24. **Yend**:                       OK           -           

In [17]:
A3 = A2[['CODE', 'WS_BODEMBREEDTE_L', 'WS_BH_BOVENSTROOMS_L', 'WS_BH_BENEDENSTROOMS_L', 'SHAPE_Length', *l_X_Y_Cols, 'geometry',]].copy()

In [18]:
A3.rename(columns={'WS_BODEMBREEDTE_L': 'width',
                  'WS_BH_BOVENSTROOMS_L': 'Elv_UStr',
                  'WS_BH_BENEDENSTROOMS_L': 'Elv_DStr',
                  'SHAPE_Length': 'length'}, inplace=True)

## 2.3. Split

### 2.3.0 Def function

In [19]:
# A3.loc[0].geometry
# A3.loc[205].geometry
# A3.loc[5460].geometry
# A3.loc[5420].geometry
# A3.loc[5974].geometry
# sindex = A3.sindex
# A3.loc[0,'CODE']
# cand_idx = list(sindex.query(A3.loc[0].geometry, predicate="intersects"))
# cand_idx = [i for i in cand_idx if i != 0]
# A3.loc[cand_idx, 'geometry'].values

In [20]:
def split_lines_by_neighbors(gdf: gpd.GeoDataFrame, tol: float = 0.000001) -> gpd.GeoDataFrame:
    """
    Split each (Multi)LineString where it meets OTHER features in gdf.
    Uses a spatial index so only intersecting neighbors are used as splitters.
    Returns one row per split segment, attributes copied from the original row.
    tol in CRS units (e.g. meters).
    """
    gdf = gdf.copy()
    geom_col = gdf.geometry.name
    other_cols = [c for c in gdf.columns if c != geom_col]
    gdf['_original_geom'] = gdf[geom_col].copy() # Store original geometries

    # Snap Cooinates so near-coincident nodes match (for spatial index and splitting)
    if hasattr(shapely, "set_precision"):
        gdf[geom_col] = shapely.set_precision(gdf[geom_col].array, tol)

    sindex = gdf.sindex

    new_rows = []

    for idx, row in gdf.iterrows():
        geom = row[geom_col]
        original_geom = row['_original_geom']
        attrs = row.to_dict()

        # candidate neighbors that intersect this geom's bbox
        cand_idx = list(sindex.query(geom, predicate="intersects"))
        cand_idx = [i for i in cand_idx if i != idx]

        if not cand_idx:
            # No neighbors - use original unsnapped geometry
            new_rows.append({**attrs, geom_col: original_geom})
            continue

        neigh_geoms = gdf.loc[cand_idx, geom_col].values

        # build splitter from neighbors
        splitter = unary_union(neigh_geoms)

        # NEW: remove overlapping segments so split() only sees crossing/touching parts
        splitter = splitter.difference(geom)

        if splitter.is_empty or not geom.intersects(splitter):
            # Has neighbors but no actual split - use original unsnapped geometry
            new_rows.append({**attrs, geom_col: original_geom})
            print(row['CODE'], "no split")
            continue

        parts = split(geom, splitter)

        # Check if geometry was actually split (more than 1 part)
        if len(parts.geoms) == 1:
            # Not actually split - use original unsnapped geometry
            new_rows.append({**attrs, geom_col: original_geom})
        else:
            # Geometry was actually split - use snapped version
            for seg in parts.geoms:
                new_rows.append({**attrs, geom_col: seg})

    # Return without _original_geom column
    return gpd.GeoDataFrame(new_rows, columns=other_cols + [geom_col], crs=gdf.crs)

### 2.3.1 Split and QA

In [21]:
A4 = split_lines_by_neighbors(A3, tol=0.001)

In [22]:
# Diagnostic: Check coordinates preservation
# Compare A3 (before split) with non-split rows in A4
A4_codes = A4['CODE'].value_counts()
non_split_codes = A4_codes[A4_codes == 1].index  # Codes that weren't split

# Get non-split rows from A4
A4_non_split = A4[A4['CODE'].isin(non_split_codes)].copy()
A3_matching = A3[A3['CODE'].isin(non_split_codes)].copy()

# Extract coordinates
A3_matching = A3_matching.set_index('CODE')
A4_non_split = A4_non_split.set_index('CODE')

# Check coordinate differences
Coo_diff = 0
for code in non_split_codes:
    if code in A3_matching.index and code in A4_non_split.index:
        g3 = A3_matching.loc[code, 'geometry']
        g4 = A4_non_split.loc[code, 'geometry']
        
        # Extract start and end coordinates properly for both LineString and MultiLineString
        if g3.geom_type == 'LineString':
            c3 = (g3.coords[0], g3.coords[-1])
        else:  # MultiLineString
            c3 = (g3.geoms[0].coords[0], g3.geoms[-1].coords[-1])
        
        if g4.geom_type == 'LineString':
            c4 = (g4.coords[0], g4.coords[-1])
        else:  # MultiLineString
            c4 = (g4.geoms[0].coords[0], g4.geoms[-1].coords[-1])
        
        if c3 != c4:
            Coo_diff += 1

print(f"Total non-split features: {len(non_split_codes)}")
print(f"Non-split features with changed coordinates: {Coo_diff}")
print(f"Total features in A3: {len(A3)}")
print(f"Total features in A4: {len(A4)}")
print(f"Features that were split: {len(A4) - len(non_split_codes)}")


Total non-split features: 12447
Non-split features with changed coordinates: 0
Total features in A3: 12864
Total features in A4: 13319
Features that were split: 872


In [23]:
l_X_Y_Cols_ = [i+'_' for i in l_X_Y_Cols]
l_Cols1 = sorted(['CODE', *l_X_Y_Cols, *l_X_Y_Cols_, 'geometry'])

In [24]:
A4[l_X_Y_Cols_] = A4.geometry.apply(
    lambda g: pd.Series(
        g.geoms[0].coords[0] + g.geoms[-1].coords[-1]
        if g.geom_type.startswith("Multi")
        else g.coords[0] + g.coords[-1],
        index=l_X_Y_Cols_
    )
)

In [25]:
A4['split'] = 0
A4_split = A4.loc[ A4['CODE'].isin(A4['CODE'].value_counts()[A4['CODE'].value_counts() > 1].index)]
A4.loc[ A4['CODE'].isin(A4['CODE'].value_counts()[A4['CODE'].value_counts() > 1].index), 'split' ] = 1

In [26]:
A4_Coo_change = A4.loc[ (A4['Xa'].round(3)  != A4['Xa_'].round(3)) | # Only contains rows where coordinates have changed
                        (A4['Ya'].round(3)  != A4['Ya_'].round(3)) | 
                        (A4['Xz'].round(3)    != A4['Xz_'].round(3)) | 
                        (A4['Yz'].round(3)    != A4['Yz_'].round(3)) ]

In [27]:
A4_split.shape, A4_Coo_change.shape, A4.shape

((872, 15), (867, 15), (13319, 15))

In [28]:
# pd.DataFrame( {'CODE': A4_split['CODE'].value_counts().index, 'counts': A4_split['CODE'].value_counts().values} ).groupby('counts').size()

In [29]:
shift = A4_Coo_change.loc[ ~A4_Coo_change['CODE'].isin(A4_split['CODE']), l_Cols1]

In [30]:
shift.shape

(0, 10)

- Shift being empty shows that no coordinates were changed, except for the CODEs that were split! As was desired.

In [31]:
# Identify rows in A4_split but not in A4_Coo_change
A4_split_no_Coo_change = A4_split.loc[~A4_split.index.isin(A4_Coo_change.index)].copy()
print(f"Rows in A4_split but not in A4_Coo_change: {len(A4_split_no_Coo_change)}")
print(f"\nUnique CODEs: {A4_split_no_Coo_change['CODE'].unique()}")
A4_split_no_Coo_change[l_Cols1]

Rows in A4_split but not in A4_Coo_change: 5

Unique CODEs: ['OVK06578' 'OVK08440' 'OVK07820' 'OVK11638' 'OVK50390']


Loading ITables v2.5.2 from the init_notebook_mode cell... (need help?)


5 entries seem to have had their changed for one of the split sub-segments but not for the other. This shows some sort of false start and going back and forth. Let's check those.

In [32]:
for i, code in A4_split_no_Coo_change['CODE'].items():
    GDF_section = A4.loc[A4['CODE'] == code, l_Cols1] # .loc['CODE'] # .iloc[i-1]
    
    for j, R in GDF_section.iterrows():
        if not np.isclose(R['Xa'], R['Xa_']) or not np.isclose(R['Ya'], R['Ya_']) or not np.isclose(R['Xz'], R['Xz_']) or not np.isclose(R['Yz'], R['Yz_']):
            print(f'Coordinates have changed for {j}, {R["CODE"]}')
            print(GDF_section)
            # print(A4.loc[j, sorted(['CODE', *l_X_Y_Cols, *l_X_Y_Cols_, 'geometry'])])
            print('-'*50)
        # A4_split_no_Coo_change.loc[i, sorted(['CODE', *l_X_Y_Cols, *l_X_Y_Cols_, 'geometry'])]

Coordinates have changed for 986, OVK06578
         CODE          Xa        Xa_       Xz       Xz_           Ya  \
986  OVK06578  86385.2477  86385.248  87035.2  86384.65  391508.1321   
987  OVK06578  86385.2477  86385.248  87035.2  87035.20  391508.1321   

            Ya_         Yz         Yz_  \
986  391508.132  392157.76  391508.098   
987  391508.132  392157.76  392157.760   

                                              geometry  
986  LINESTRING (86385.248 391508.132, 86384.65 391...  
987  LINESTRING (86385.248 391508.132, 86387.532 39...  
--------------------------------------------------
Coordinates have changed for 4570, OVK08440
          CODE          Xa         Xa_          Xz         Xz_          Ya  \
4569  OVK08440  105006.114  105006.114  104305.596  104305.596  405299.683   
4570  OVK08440  105006.114  104305.596  104305.596  104305.490  405299.683   

             Ya_          Yz         Yz_  \
4569  405299.683  405293.303  405293.303   
4570  405293.303  405293

In [33]:
A4.loc[ A4['CODE'].isin(A4_split_no_Coo_change['CODE']), l_X_Y_Cols_ ] = A4.loc[ A4['CODE'].isin(A4_split_no_Coo_change['CODE']), l_X_Y_Cols ].values

Something weird is happending to those 5. I'll proceed as is for now. Maybe later I'll come back to it. # 666

### 2.3.2 Compare python split against QGIS split

I noticed that there are fewer rows in A4 than the split layer in QGIS. So let's check which ones are missing.

In [34]:
B = gpd.read_file(r'g:\models\NBr\PrP\SFR\BrabantseDelta\Gpkg\Ss\WBD_1ry_SW_NW_split.gpkg')

In [35]:
A4.shape, B.shape, B.shape[0] - A4.shape[0]

((13319, 15), (13751, 56), 432)

In [36]:
B_split = B.loc[ B['CODE'].isin(B['CODE'].value_counts()[B['CODE'].value_counts() > 1].index) ]

In [37]:
i = 1
for j in A4_split['CODE'].unique():
    if j not in B_split['CODE'].values:
        print(i, j)
        i += 1

1 OVK03452
2 OVK08249
3 OVK10775
4 OVK05598
5 OVK01467
6 OVK02811
7 OVK07817
8 OVK10774
9 OVK11186
10 OVK00395
11 OVK04879
12 OVK04280
13 OVK06504
14 OVK05884
15 OVK20912
16 OVK20993
17 OVK12538
18 OVK20937
19 OVK00302
20 OVK10552
21 OVK10778
22 OVK04401
23 OVK06177
24 OVK00256
25 OVK07165
26 OVK21134
27 OVK11948
28 OVK22025
29 OVK21792
30 OVK01530
31 OVK12398
32 OVK00022
33 OVK21211
34 OVK21092
35 OVK11276


I checked the 1st 3 codes that are missing in B_split. The splitting performed in A4, although sometimes pointless in practice, makes sense based on our splitting function.

Now let's check the opposite: which codes are in A4_split but not in B_split.

In [38]:
i = 1
for j in B_split['CODE'].unique():
    if j not in A4_split['CODE'].values:
        print(i, j)
        i += 1

1 OVK10848
2 OVK12073
3 OVK12577
4 OVK08118
5 OVK04889
6 OVK08557
7 OVK05472
8 OVK21770
9 OVK01502
10 OVK21923
11 OVK08190
12 OVK12022
13 OVK05620
14 OVK21003
15 OVK00667
16 OVK11295
17 OVK09411
18 OVK09164
19 OVK07859
20 OVK12111
21 OVK12277
22 OVK09985
23 OVK00927
24 OVK04492
25 OVK11669
26 OVK04305
27 OVK04231
28 OVK09545
29 OVK04845
30 OVK00447
31 OVK01089
32 OVK05988
33 OVK09601
34 OVK05860
35 OVK04660
36 OVK09224
37 OVK09195
38 OVK05862
39 OVK09030
40 OVK09469
41 OVK06114
42 OVK00139
43 OVK06181
44 OVK03564
45 OVK06256
46 OVK11950
47 OVK11915
48 OVK06464
49 OVK07353
50 OVK05621
51 OVK09125
52 OVK08857
53 OVK00470
54 OVK00511
55 OVK10538
56 OVK00502
57 OVK08254
58 OVK00979
59 OVK01644
60 OVK10023
61 OVK07368
62 OVK05956
63 OVK06664
64 OVK08377
65 OVK07321
66 OVK06967
67 OVK21874
68 OVK07480
69 OVK03457
70 OVK00319
71 OVK20511
72 OVK11086
73 OVK05583
74 OVK00659
75 OVK02772
76 OVK11177
77 OVK02571
78 OVK09981
79 OVK07395
80 OVK06769
81 OVK07183
82 OVK00402
83 OVK10163
84 OVK06406
8

1. doesn't make sense. It's one segment that's not intersected by another (besides the ends).
2. doesn't make sense either. Same as above.
3. same for this one.

A4 seems more reliable. We'll proceed with it and move the QGIS split file to a Ss folder.

### 2.3.3 Recalc/assign some columns

In [39]:
A4_split['CODE'].value_counts()

Loading ITables v2.5.2 from the init_notebook_mode cell... (need help?)


#### Length

In [40]:
A4.loc[A4['CODE']=='OVK03536']

Loading ITables v2.5.2 from the init_notebook_mode cell... (need help?)


Length needs to be recalculated after the split, based on the new geometries. As you can see above, all lengths for split segments are currently the same as the original segment length.

In [41]:
A5 = A4.copy()

In [42]:
A5['length'] = A5.geometry.length

In [43]:
A5.loc[A4['CODE']=='OVK03536', 'length']

Loading ITables v2.5.2 from the init_notebook_mode cell... (need help?)


In [44]:
A5.loc[A4['CODE']=='OVK03536', 'length'].sum().round(3), A4.loc[A4['CODE']=='OVK03536','length'].iloc[0].round(3)

(np.float64(621.801), np.float64(621.99))

The length sum isn't exactly identical, but I'll move forward for now. #666 can come back to this later

#### Coordinates

In [45]:
Bool_Coo_changed = (A5[l_X_Y_Cols].sum(axis=1)!=A5[l_X_Y_Cols_].sum(axis=1))

In [46]:
A5.loc[Bool_Coo_changed, l_X_Y_Cols] = A5.loc[Bool_Coo_changed, l_X_Y_Cols_].values

In [47]:
(A5.loc[:, l_X_Y_Cols].values == A5.loc[:, l_X_Y_Cols_].values).all()

np.True_

All values from the newly calculated coordinates have been assigned to the main GDF.

In [48]:
A5.drop(columns=l_X_Y_Cols_ , inplace=True)

In [49]:
A5.loc[ A5['split']==1, ['CODE', 'Elv_UStr', 'Elv_DStr'] ]

Loading ITables v2.5.2 from the init_notebook_mode cell... (need help?)


## 2.4. Calculate routing

### 2.4.0 Create ID Col & Identify downstream

In [50]:
A5.columns

Index(['CODE', 'width', 'Elv_UStr', 'Elv_DStr', 'length', 'Xa', 'Ya', 'Xz',
       'Yz', 'geometry', 'split'],
      dtype='object')

In [51]:
A5['ID'] = range(1, len(A5)+1)

In [52]:
# Create a lookup dictionary from start coordinates to CODE
Coo_to_id = {(R.Xa, R.Ya): (R.CODE, R.ID) for R in A5.itertuples()}

print(f"✓ Lookup dictionary created with {bold}{len(Coo_to_id)}{style_reset} entries.")

✓ Lookup dictionary created with 12539 entries.


In [53]:
# Function to find the downstream ID
def get_DStr(row):
    end_Coos = (row.Xz, row.Yz)
    result = Coo_to_id.get(end_Coos, (0, 0))
    return result

# Apply the function to create the 'DStr' column
A5[['DStr_code', 'DStr_ID']] = A5.apply(get_DStr, axis=1, result_type='expand')

print("✓ 'DStr' columns calculated.")

✓ 'DStr' columns calculated.


In [54]:
print(f'{round(A5['DStr_code'].value_counts().max()/A5.shape[0]*100, 2)} % of DStrs are 0 (i.e. no start Cooinates match the end Cooinates of the current node).')

14.15 % of DStrs are 0 (i.e. no start Cooinates match the end Cooinates of the current node).


The percentage is much bigger than expected. Let's investigate.

### 2.2.1 Investigate segments that failed to connect

#### Check out number of matches/no matches

In [55]:
A5['DStr_match'] = A5['DStr_code'].isin(A5['CODE'])
A5['DStr_code'].value_counts()

Loading ITables v2.5.2 from the init_notebook_mode cell... (need help?)


It makes sense that most nodes have just a few upstream nodes. 1353 nodes are not connected.

In [56]:
A5['DStr_match'].value_counts()

Loading ITables v2.5.2 from the init_notebook_mode cell... (need help?)


#### Calculate min distance from start to any reach's end and investigate no matches.

In [57]:
A5['min_Dist'] = 0.0
A5.loc[A5['DStr_code'] == 0, 'min_Dist'] = A5.loc[A5['DStr_code'] == 0].apply(lambda row: C.c_Dist(row['Xz'], row['Yz'], A5['Xa'], A5['Ya']).min(), axis=1)

In [58]:
N_total_no_match = (A5['DStr_code'] == 0).sum()

In [59]:
A5.loc[A5['DStr_match']==False,'min_Dist'].describe()

Loading ITables v2.5.2 from the init_notebook_mode cell... (need help?)


In [60]:
l_Vals = [0.001, 0.1, 1, 10, 100, 1000, 10000]

print(f'Out of the {N_total_no_match} segments that do not match:')

N_below_Prv, Val_Prv = 0, 0
for v in l_Vals:
    N_below = (A5.loc[A5['DStr_match']==False,'min_Dist'] <= v).sum()
    P_below = round(N_below / N_total_no_match * 100, 2)
    
    sample_A5 = A5.loc[(A5['min_Dist'] > Val_Prv) & (A5['min_Dist'] <= v), ['CODE', 'min_Dist']].sort_values(by='min_Dist')
    sample_A5['Code:min_Dist'] = sample_A5.apply(lambda row: f"{row['CODE']}: {row['min_Dist']:8.4f}", axis=1)
    sample_A5 = sample_A5['Code:min_Dist']
    example_nodes = sample_A5.iloc[:].tolist()

    print(f'-{Val_Prv:6} < min_Dist <= {v:5} |N: {N_below:6} (+ {(N_below - N_below_Prv):4}) ({round(P_below,1):5} %) | Codes: {example_nodes}\n')

    N_below_Prv, Val_Prv = N_below, v

Out of the 1884 segments that do not match:
-     0 < min_Dist <= 0.001 |N:   1526 (+ 1526) ( 81.0 %) | Codes: ['OVK08923:   0.0000', 'OVK11912:   0.0000', 'OVK04409:   0.0000', 'OVK11594:   0.0000', 'OVK08796:   0.0000', 'OVK02198:   0.0000', 'OVK11798:   0.0000', 'OVK03488:   0.0000', 'OVK03489:   0.0000', 'OVK07542:   0.0000', 'OVK05267:   0.0000', 'OVK07868:   0.0000', 'OVK06616:   0.0000', 'OVK03461:   0.0000', 'OVK10061:   0.0000', 'OVK00162:   0.0000', 'OVK08056:   0.0000', 'OVK12116:   0.0000', 'OVK20930:   0.0000', 'OVK12123:   0.0000', 'OVK00463:   0.0000', 'OVK05975:   0.0000', 'OVK03034:   0.0000', 'OVK11335:   0.0000', 'OVK06795:   0.0000', 'OVK10098:   0.0000', 'OVK04344:   0.0000', 'OVK10040:   0.0000', 'OVK11655:   0.0000', 'OVK12498:   0.0000', 'OVK04099:   0.0000', 'OVK02337:   0.0000', 'OVK00570:   0.0000', 'OVK08058:   0.0000', 'OVK08966:   0.0000', 'OVK03190:   0.0000', 'OVK03525:   0.0000', 'OVK12048:   0.0000', 'OVK03168:   0.0000', 'OVK11700:   0.0000', 'OVK0059

The total number of segments that do not match is too high to check them all, but we'll check some cases.

- The ones <1m can be attributed to closing errors, and we can connect them to the closest one via an algorithm.
- They take up 86% of the unmatched, which means there aren't that many remaining.
- Let's print out the names of the bigger differences, so that we can check them in QGIS.
    - Some of the smallest min_Dist values, e.g. OVK08923 are clearly closing errors.
    - Checking some fo the biggest values <1m, e.g. OVK09435, OVK05835, OVK02702, shows that they are also closing errors.

In [61]:
A5.loc[A5['min_Dist'].between(1,10, inclusive='right'), ['CODE', 'min_Dist']].sort_values(by='min_Dist').reset_index(drop=True)

Loading ITables v2.5.2 from the init_notebook_mode cell... (need help?)


- Some of the 1-10m differences, e.g. OVK06539, OVK08754, OVK21755, should have been connected to a downstream segment, but they were not.
- Others, like OVK05556, OVK00509, lead outside the area of the model.
- We can't check them all now, I'll only check those inside the model area that I'm innterested in.

In [ ]:
A5_within = A5.loc[A5.within(box(Xmin, Ymin, Xmax, Ymax))]

In [79]:
A5_within.loc[ A5_within['min_Dist'].between(0,1, inclusive='right')]

Loading ITables v2.5.2 from the init_notebook_mode cell... (need help?)


In [78]:
A5_within.loc[ A5_within['min_Dist'].between(1,10000, inclusive='right')]

Loading ITables v2.5.2 from the init_notebook_mode cell... (need help?)


There are also no items in our region with distances between 1m and 10km. We'll ignore those for now. Further analysis can be performed later, if there is interest in expanding the region. #666

### 2.2.2 Edit connections

In [80]:
# Initialize the 'multiple_close' column with empty strings
A5['multiple_close'] = ""

# Select rows to correct
rows_to_correct = A5[(A5['DStr_code'] == 0) & (A5['min_Dist'] < 1)].copy()

print(f"Found {len(rows_to_correct)} segments with no downstream connection and a potential connection within 1m.")

corrected_count = 0
multiple_count = 0

# Loop through the rows that need correction
for i, R in rows_to_correct.iterrows():
    # Calculate distances from the current row's end point to all other rows' start points
    distances = C.c_Dist( R['Xz'], R['Yz'], A5['Xa'], A5['Ya'])
    
    # Find segments where the distance is less than 1m
    close_mask = (distances < 1) & (A5.index != i) # Exclude self
    close_segments_codes = A5['CODE'][close_mask].tolist()
    close_segments_object_ids = A5['ID'][close_mask].tolist()
    
    if len(close_segments_codes) == 1:
        # If there is exactly one close segment, update both 'DStr_code' and 'DStr_ID'
        A5.loc[i, 'DStr_code'] = close_segments_codes[0]
        A5.loc[i, 'DStr_ID'] = close_segments_object_ids[0]
        corrected_count += 1
    elif len(close_segments_codes) > 1:
        # If there are multiple close segments, store as comma-separated string
        A5.loc[i, 'multiple_close'] = ", ".join(close_segments_codes)
        multiple_count += 1

print(f"✓ Corrected {corrected_count} segments by updating 'DStr_code' and 'DStr_ID'.")
print(f"✓ Found {multiple_count} segments with multiple potential connections (stored in 'multiple_close').")

Found 1621 segments with no downstream connection and a potential connection within 1m.
✓ Corrected 1512 segments by updating 'DStr_code' and 'DStr_ID'.
✓ Found 109 segments with multiple potential connections (stored in 'multiple_close').
✓ Corrected 1512 segments by updating 'DStr_code' and 'DStr_ID'.
✓ Found 109 segments with multiple potential connections (stored in 'multiple_close').


In [83]:
A5_within = A5.loc[A5.within(box(Xmin, Ymin, Xmax, Ymax))]

In [84]:
A5_within.loc[A5_within['multiple_close']!='', ['CODE', 'multiple_close']].sort_values(by='multiple_close', ascending=False)

Loading ITables v2.5.2 from the init_notebook_mode cell... (need help?)


I'll manually conect those to the correct downstream segment.

In [86]:
A5.loc[A5['CODE']=='OVK20460', ['DStr_code', 'DStr_ID']] = ['OVK20461', A5.loc[A5['CODE']=='OVK20461', 'ID'].values[0]]

In [87]:
A5.loc[A5['CODE']=='OVK00976', ['DStr_code', 'DStr_ID']] = ['OVK01474', A5.loc[A5['CODE']=='OVK01474', 'ID'].values[0]]

## 2.4 Correct Elv_UStr and Elv_DStr

In [91]:
l_A5_Cols = ['ID', 'CODE', 'width', 'length', 'Elv_UStr', 'Elv_DStr', 'DStr_code', 'DStr_ID', 'Xa', 'Ya', 'Xz', 'Yz', 'geometry', 'split']

In [94]:
A5.loc[A5['split']==1, l_A5_Cols]

Loading ITables v2.5.2 from the init_notebook_mode cell... (need help?)


## 2.4. Explore GDF to fill SFRmaker input requirements

### 2.4.0 Custom Hydrography Required Fields

Below, we'll match the shapefile attributes to the SFRmaker requirements as per SFRmaker manual ( https://doi-usgs.github.io/sfrmaker/latest/inputs.html):

**Custom hydrography** <br>
Any Polyline shapefile can be supplied in lieu of NHDPlus, but it must have the following columns, as shown in the second example:
1. **flowlines_file**: path to shapefile
2. **id_column**: unique identifier for each polyline
3. **routing_column**: downstream connection (ID), 0 if none
4. **width1_column**: channel width at start of line, in attr\_length\_units (optional)
5. **width2_column**: channel width at end of line, in attr_length_units (optional)
6. **up_elevation_column**: streambed elevation at start of line, in attr_height_units
7. **dn_elevation_column**: streambed elevation at end of line, in attr_height_units
8. **name_column**: stream name (optional)
9. **attr_length_units**: channel width units
10. **attr_height_units**: streambed elevation units

Here are the columns of the GDF, so we can match them to the SFRmaker requirements:

In [ ]:
for i, col in enumerate(GDF.columns):
    print(f"{i}: {col}")

### 2.4.2 Explore GDF fields

#### 2. & 3. will most likely be 'OBJECTID' & 'DStr_ID'.

But let's ensure there are no nulls and use the .describe() method to check more details

In [ ]:
GDF[['OBJECTID', 'DStr_ID']].isna().sum()

In [ ]:
GDF[['CODE', 'OBJECTID', 'DStr_code', 'DStr_ID']].describe(include='all')

In [ ]:
GDF.loc[GDF['DStr_ID']==0, 'DStr_ID'].count()

Great. No nuls and 13 DStr_codes with no downstream connection ('DStr'==0) (9 with min_Dist between 100 and 1000, + 5 with min_Dist betwen 10 and 100, -1, that we connected to the meandering long segment).

#### 4. & 5. Widths (at start and end)

There are 4 candidates for those fields:
- 'WS_BODEMBREEDTE_L' (channel bed width)
- 'WS_BODBREE_ACCPROF_LI_L' (Accepted profile bottom width (left) (m))
- 'WS_BODBREE_ACCPROF_RE_L' (Accepted profile bottom width (right) (m))
- 'WS_VW_BODEMBREEDTE_L' (Prescribed bottom width (voorwaarde) (m))

Let's compare the values. We can probably just use WS_BODEMBREEDTE. I think it has no/fewer nuls and the values are similar to the other 3 fields. Then we can upgrade. But let's have a look.

In [ ]:
GDF[['CODE', 'OBJECTID', 'WS_BODEMBREEDTE_L', 'WS_BODBREE_ACCPROF_LI_L', 'WS_BODBREE_ACCPROF_RE_L', 'WS_VW_BODEMBREEDTE_L']].describe(include='all')

In [ ]:
GDF.loc[ GDF['WS_BODBREE_ACCPROF_LI_L'].notna() | GDF['WS_BODBREE_ACCPROF_RE_L'].notna() | GDF['WS_VW_BODEMBREEDTE_L'].notna(),
         ['CODE', 'WS_BODEMBREEDTE_L', 'WS_BODBREE_ACCPROF_LI_L', 'WS_BODBREE_ACCPROF_RE_L', 'WS_VW_BODEMBREEDTE_L']]

'WS_BODBREE_ACCPROF_LI_L' & 'WS_BODBREE_ACCPROF_RE_L' are not the bottom of the channel. This is clear if we masure the channel width in QGIS. So we won't use those.<br>
'WS_VW_BODEMBREEDTE_L' only has nulls, so we won't use that either.<br>
**So we'll just use 'WS_BODEMBREEDTE_L'.**

#### 6. & 7. Upstream and downstream elevations

In [ ]:
GDF[[ 'CODE', 'WS_BH_BOVENSTROOMS_L', 'WS_BH_BENEDENSTROOMS_L']].describe(include='all')

No nulls + the percentiles make sense.🟢<br>
Let's make sure the UStr is always higher than the DnStr.<br>
Then let's print out some values to check in QGIS.

In [ ]:
(GDF['WS_BH_BOVENSTROOMS_L'] <= GDF['WS_BH_BENEDENSTROOMS_L']).sum(), (GDF['WS_BH_BOVENSTROOMS_L'] < GDF['WS_BH_BENEDENSTROOMS_L']).sum(), (GDF['WS_BH_BOVENSTROOMS_L'] > GDF['WS_BH_BENEDENSTROOMS_L']).sum(), GDF.shape[0]

For 98 segments (out of 591), the UStr Elv is <= the DStr Elv. This is not good. We'll have to fix this. 🔴<br>
Only 24/99 have DStr < UStr , the rest are equal. = will be corrected by SFR itself (as far as I know), so no action is required for those.

##### Let's print out some CODEs where =, to check in QGIS. *(We don't really need to, I'm just curious)*

In [ ]:
GDF_Elv = GDF[['CODE', 'WS_BH_BOVENSTROOMS_L', 'WS_BH_BENEDENSTROOMS_L', 'DStr_code', 'DStr_ID']].copy()

In [ ]:
GDF_Elv['Diff'] = GDF_Elv['WS_BH_BOVENSTROOMS_L'] - GDF_Elv['WS_BH_BENEDENSTROOMS_L']

In [ ]:
GDF_Elv.loc[GDF_Elv['Diff'] == 0].head()

In [ ]:
GDF_Elv.loc[ GDF_Elv['Diff'] < 0 ].sort_values(by='Diff', ascending=True).head()

##### Let's see if any of the problematic segments have multiple UStr segments. That would make a solution harder to implement.<br>
*(if there is only 1 UStr segment, the DStr Elv of the UStr segment can be modified to allow the UStr Elv of the current segmet to be increased as well, but if there are multiple, this becomes more complicated)*

In [ ]:
l_problematic = GDF_Elv.loc[ GDF_Elv['Diff'] < 0, 'CODE'].tolist()
for S in l_problematic:
    sum = (GDF['DStr_code']==S).sum()
    if sum > 1:
        print(S, sum)

There are multiple segments with more than 1 UStr segment. We'll have to consider this when designing the elevation correction logic. 

##### Elv correction algorithm

We'll design an algorithm to fix those with <. Those with = will be fixed by SFR itself (hopefully). The following abbreviations are useful for explaining the concept:
- A: DStr Elv of DStr segment
- B: UStr Elv of DStr segment
- C: DStr Elv of current segment
- D: UStr Elv of current segment
- F: DStr Elv of UStr segment(s)

Here is the idea behind the algorithm:
1. If **C > D & B <= D** :<br>
-> Set **C = D**
2. If **C > D & B > D** :<br>
-> Set **C = D**. Set **B = D**
3. If **C <= D** :<br>
-> **No action**.

Repeat till there are no segments with C < D.

When there is no downstream segment, we apply the logic used in case 1.

In [ ]:
GDF_Elv = GDF_Elv.merge(GDF[['CODE', 'WS_BH_BOVENSTROOMS_L', 'WS_BH_BENEDENSTROOMS_L']], left_on='DStr_code', right_on='CODE', suffixes=('', '_DStr'), how='left')

In [ ]:
GDF_Elv[['A', 'B']] = GDF_Elv[['WS_BH_BENEDENSTROOMS_L_DStr', 'WS_BH_BOVENSTROOMS_L_DStr']].copy()

In [ ]:
GDF_Elv[['C', 'D']] = GDF_Elv[['WS_BH_BENEDENSTROOMS_L', 'WS_BH_BOVENSTROOMS_L']].copy()

In [ ]:
GDF_Elv[GDF_Elv['B'].isna()]

In [ ]:
init = GDF_Elv.loc[ GDF_Elv['CODE']=='OVK02121', 'B' ].values[0]

In [ ]:
def adjust_elevations(row):
    if row['C'] <= row['D']: # If UStr Elv <= DStr Elv, no adjustment needed
        return row['B'], row['C']
    elif (row['C'] > row['D']) and (pd.isna(row['B'])): # If UStr Elv <= DStr Elv but DStr Elv is missing (OuFl segment), set both to DStr Elv
        return pd.NA, row['D']
    elif (row['C'] > row['D']) and (row['B'] <= row['D']):
        return row['B'], row['D']
    elif (row['C'] > row['D']) and (row['B'] > row['D']):
        return row['D'], row['D']
    else:
        # Default case - should not happen, but ensures function always returns a tuple
        return row['B'], row['C']

In [ ]:
GDF_Elv[['B_', 'C_']] = GDF_Elv.apply(adjust_elevations, axis=1, result_type='expand')

I'm worried consequtive segments might be problematic. Let's check if there are any.

In [ ]:
GDF_Elv_unfixed = GDF_Elv[ (GDF_Elv['Diff']<0)]
consequtive = GDF_Elv_unfixed.loc[GDF_Elv_unfixed['DStr_code'].isin(GDF_Elv_unfixed['CODE']), 'DStr_code']
GDF_Elv_unfixed.loc[ (GDF_Elv_unfixed['CODE'].isin(consequtive)) | (GDF_Elv_unfixed['DStr_code'].isin(consequtive)), ['CODE', 'DStr_code', 'A', 'B', 'B_', 'C', 'C_', 'D']].sort_values(by='D').reset_index(drop=True)

Consequtive ok. Let's check if there is a drop in Elv (positive slope) for each segment.

In [ ]:
GDF_Elv.loc[ GDF_Elv['D'] - GDF_Elv['C_'] < 0 ]

Cool, no segments without any drop in Elv.

In [ ]:
GDF_Elv['segment_drop'] = GDF_Elv['D'] - GDF_Elv['C_']
GDF_Elv['DStr_drop'] = GDF_Elv['C_'] - GDF_Elv['B']
GDF_Elv.loc[ GDF_Elv['C_'] - GDF_Elv['B_'] < 0 , ['CODE', 'DStr_code', 'A', 'B', 'B_', 'C', 'C_', 'D', 'segment_drop', 'DStr_drop'] ].sort_values(by='DStr_drop').reset_index(drop=True) 

There are **quite a few** segments where C_ > B!!! SFRmaker might fix this. If not, I'll come back and fix it.

#### Conclusion

**Custom hydrography** <br>
Any Polyline shapefile can be supplied in lieu of NHDPlus, but it must have the following columns, as shown in the second example:
1. flowlines_file: path to shapefile <br>
    -> **Pa_GPkg_1ry** <br>
2. id_column: unique identifier for each polyline <br>
    -> **OBJECTID** <br>
3. routing_column: downstream connection (ID), 0 if none <br>
    -> **DStr_ID** <br>
4. width1_column: channel width at start of line, in attr\_length\_units (optional) <br>
    -> **WS_BODEMBREEDTE_L** <br>
5. width2_column: channel width at end of line, in attr_length_units (optional) <br>
    -> **WS_BODEMBREEDTE_L** <br>
6. up_elevation_column: streambed elevation at start of line, in attr_height_units <br>
    -> **WS_BH_BOVENSTROOMS_L** <br>
7. dn_elevation_column: streambed elevation at end of line, in attr_height_units <br>
    -> **WS_BH_BENEDENSTROOMS_L** <br>
8. name_column: stream name (optional) <br>
    -> **CODE** <br>
9. attr_length_units: channel width units <br>
    -> **'m'** <br>
10. attr_height_units: streambed elevation units <br>
    -> **'m'** <br>

In [ ]:
GDF_Elv

In [ ]:
GDF_copy = GDF.copy()

In [ ]:
GDF = GDF_copy.copy()

In [ ]:
GDF = GDF.merge( GDF_Elv[['CODE', 'C_', 'D']], on='CODE', how='left')

## 2.5 Generate SFRmaker lines

In [ ]:
GDF['width2'] = GDF['WS_BODEMBREEDTE_L'].copy()

In [ ]:
lines = sfr.Lines.from_dataframe(df=GDF.copy(), # .copy() to avoid GDF columns being renamed by function (this feels like a bug to me)
    id_column='OBJECTID',
    routing_column='DStr_ID',
    width1_column='WS_BODEMBREEDTE_L', width2_column='width2',
    dn_elevation_column='C_',
    up_elevation_column='D',
    name_column='CODE',
    width_units='m',
    height_units='m',
    crs=GDF.crs
    #    shapefile=Pa_GPkg_1ry_SHP_SFR,
)

In [ ]:
DF_lines = lines.df
U.DF_info(lines.df)

# -1. Junkyard

In [ ]:
# ### 2.2.3 Check CODE
# #### Uniqueness
# **This section was designed before the shapefiles were split. It's not valid anymore, hence most of the cells have been removed.**
# There shouldn't be any duplicates. Let's check those codes in QGIS.
# After checking in QGIS, I see that the longer OWL17672 item doesn't make sense. I'll simply remove it.
# GDF_detail.drop(GDF_detail[GDF_detail['CODE'] == 'OWL17672'].sort_values('SHAPE_Length').iloc[:1].index, inplace=True)
# ##### Check combined
# #### Codes Missing from detailed GDF
# A.loc[~A['CODE'].isin(GDF_detail['CODE'].values)]
# This is strange, I was expecting the detailed GDF to have all the codes from A, but their number is small. I'll check for those items on the map
# - **OVK02041**: is a small ditch, that for whatever reason isn't part of GDF_detail (there is nothing in that exact spot, but it's downstream segment is there). 
# - **OVK00911**: is the end part of OWL37987 in detail.
# - **OVK02271**, **OVK11550**, **OVK02270**: All 3 of those form a big segmet in detailed, but they're split in 1ry.
# ### 2.2.4 Merge
# GDF = pd.merge(GDF_detail, A, on='CODE', how='outer')  # was how='all'
# GDF_ = GDF.copy()
# GDF_detail.shape, A.shape, GDF.shape
# GDF[sorted([i for i in GDF.columns if 'geometry' not in i])].sort_values('WS_BH_BOVENSTROOMS')
# ### 2.2.5 Check
# #### Start
# Let's check if the _x, _y columns are identical. If yes, we can drop one of them.
# [i for i in GDF.columns if '_x' in i ]
# We're expecting 591 matches, as that's the number of values in A, and the same info was copied to GDF_detail.
# # def compare_GDF_columns(GDF, col_x, col_y=None):
# #     if type(col_x) is not list:
# #         col_x = [col_x]
# #     if not col_y:
# #         col_y = [i.replace('_x', '_y') for i in col_x]
# #     if type(col_y) is not list:
# #         col_y = [col_y]

# #     if all(i in GDF.columns for i in col_x) and all(i in GDF.columns for i in col_y):
# #         either_not_na = GDF[col_x + col_y].notna().any(axis=1)
# #         comparison = np.isclose(GDF.loc[either_not_na, col_x], GDF.loc[either_not_na, col_y], equal_nan=True).all(axis=1)
# #         num_equal = comparison.sum()
# #         print(f"Columns '{col_x, col_y}': {num_equal} values are equal.\nHere are the values that aren't equal:")

# #         display(GDF.loc[either_not_na].loc[ ~comparison, ['CODE', *col_x, *col_y]])
# #     else:
# #         print(f"Columns '{col_x, col_y}' not in GDF.columns.")
# def compare_GDF_columns(GDF, col_x, col_y=None, rtol=1e-5, hide_x_na=False, hide_y_na=False, atol=None):
#     if not col_y:
#         col_y = col_x.replace('_x', '_y')

#     if col_x in GDF.columns and col_y in GDF.columns:
#         l_na_to_remove = [col_x, col_y]
#         if hide_x_na: l_na_to_remove.remove(col_y)
#         if hide_y_na: l_na_to_remove.remove(col_x)

#         either_not_na = GDF[l_na_to_remove].notna().any(axis=1)
#         if atol:
#             comparison = np.isclose(GDF.loc[either_not_na, col_x], GDF.loc[either_not_na, col_y], atol=atol, equal_nan=True)
#         else:
#             comparison = np.isclose(GDF.loc[either_not_na, col_x], GDF.loc[either_not_na, col_y], rtol=rtol, equal_nan=True)
#         num_equal = comparison.sum()
#         print(f"Columns '{col_x, col_y}': {num_equal} values are equal.\nHere are the values that aren't equal:")
#         DF = GDF.loc[either_not_na].loc[ ~comparison, ['CODE', col_x, col_y]]
#         return DF
#     else:
#         print(f"Columns '{col_x, col_y}' not in GDF.columns.")
# GDF.shape, A.shape, GDF_detail.shape
# GDF[sorted(GDF.columns)].describe()# diagnostic for merge mismatch
# print("A shape:", A.shape)
# print("GDF_detail shape:", GDF_detail.shape)
# print("GDF (merged) shape:", GDF.shape)
# print()

# # counts / uniqueness
# print("A CODE unique:", A['CODE'].nunique(), "rows with duplicate CODE:", A.shape[0] - A['CODE'].nunique())
# print("GDF_detail CODE unique:", GDF_detail['CODE'].nunique(), "rows with duplicate CODE:", GDF_detail.shape[0] - GDF_detail['CODE'].nunique())
# print()

# # check for NaNs in CODE
# print("NaNs in CODE -- A:", A['CODE'].isna().sum(), "  GDF_detail:", GDF_detail['CODE'].isna().sum())
# print()

# # how many non-nulls in the merged column the user noticed
# print("Non-null count of SHAPE_Length_y in GDF:", GDF['SHAPE_Length_y'].notna().sum())

# # find codes that produce multiple rows in the merged result
# dup_codes = GDF.groupby('CODE').size().sort_values(ascending=False)
# print("Top group sizes in merged GDF (CODE -> rows):")
# print(dup_codes.head(10))
# print()

# # list codes that appear more than once in merged result
# multi = dup_codes[dup_codes > 1]
# if not multi.empty:
#     print("Codes that yield >1 merged rows (first 30):")
#     print(multi.head(30))
#     # show all merged rows for one example code
#     example = multi.index[0]
#     print("\nExample expanded rows for CODE =", example)
#     display(GDF.loc[GDF['CODE'] == example])
# else:
#     print("No CODE with multiple merged rows found")

# # check duplicates in the source dfs
# print("\nDuplicates in A (CODEs and counts >1):")
# print(A['CODE'].value_counts().loc[lambda s: s>1].head())
# print("\nDuplicates in GDF_detail (CODEs and counts >1):")
# print(GDF_detail['CODE'].value_counts().loc[lambda s: s>1].head())

# # Optional: try merge validation to catch relationship type
# try:
#     pd.merge(GDF_detail[['CODE']].drop_duplicates(), A[['CODE']].drop_duplicates(), on='CODE', validate='one_to_one')
#     print("\nvalidate=one_to_one: OK (both sides unique after drop_duplicates).")
# except Exception as e:
#     print("\nvalidate=one_to_one raised:", e)
# GDF[sorted(GDF.columns)].describe()
# The numbers make sense. We have 5 more rows in the GDF than in GDF_detail, which makes sense as we found that 5 codes were missing from GDF_detail.
# #### - SHAPE_Length
# length_comparison = compare_GDF_columns(GDF, 'SHAPE_Length_x', rtol=0.01, hide_y_na=True)
# length_comparison
# # Find CODEs with multiple rows where SHAPE_Length_y values differ
# duplicated_codes = length_comparison[length_comparison['CODE'].duplicated(keep=False)]
# codes_with_diff_lengths = []

# for code in duplicated_codes['CODE'].unique():
#     code_rows = length_comparison[length_comparison['CODE'] == code]
#     # Check if SHAPE_Length_y values are NOT all identical
#     if code_rows['SHAPE_Length_y'].nunique() > 1:
#         codes_with_diff_lengths.append(code)

# if codes_with_diff_lengths:
#     print(f"Found {len(codes_with_diff_lengths)} CODEs with identical CODE but different SHAPE_Length_y values:")
#     result = length_comparison[length_comparison['CODE'].isin(codes_with_diff_lengths)].sort_values('CODE')
#     display(result)
# else:
#     print("No CODEs found with identical CODE but different SHAPE_Length_y values")
# length_comparison.loc[length_comparison['SHAPE_Length_x'].isna()]
# All features where Shape_Length_y is a value and Shape_Length_x is NaN will be removed for simplicity, as there are features coming from GDF_detail that already represent them. This could be improved later, as the GDF_detail features lack some info, but for now this will do.<br>
# OVK02041 doesn't have a duplicate, but will also be removed as it's deemed unimportant.
# GDF.drop(GDF.loc[GDF['SHAPE_Length_x'].isna()].index, inplace=True)
# _y length seems more accurate, but features are not split. This means _y uses the aggregate length values for the CODEs that are split in _y, which is incorrect. 

# Thus, we'll keep _x length values.
# GDF['Length'] = GDF['SHAPE_Length_x'].combine_first(GDF['SHAPE_Length_y'])
# #### - Boven/Benedenstrooms
# compare_GDF_columns(GDF, 'WS_BH_BOVENSTROOMS', 'WS_BH_BOVENSTROOMS_L')
# compare_GDF_columns(GDF, 'WS_BH_BENEDENSTROOMS', 'WS_BH_BENEDENSTROOMS_L')
# - There are some differences. I've already used WS_BH_BOVENSTROOMS_L and WS_BH_BENEDENSTROOMS_L in the SFRmaker script, so I'll keep those for now. Wherever these have gaps, we'll fill them from WS_BH_BOVENSTROOMS and WS_BH_BENEDENSTROOMS. The devil you know is better than the one you don't.
# - But those have some 99 values, which I don't like (they're too round and a bit over the elevation of our region). We'll remove those after.
# sorted(GDF.columns)
# GDF['Elv_UStr'] = GDF['WS_BH_BOVENSTROOMS_L'].combine_first(GDF['WS_BH_BOVENSTROOMS'])
# GDF['Elv_DStr'] = GDF['WS_BH_BENEDENSTROOMS_L'].combine_first(GDF['WS_BH_BENEDENSTROOMS'])
# # either_not_na = GDF['WS_BH_BENEDENSTROOMS'].notna() | GDF['WS_BH_BENEDENSTROOMS_L'].notna()
# # comparison = np.isclose(GDF.loc[either_not_na, 'WS_BH_BOVENSTROOMS'], GDF.loc[either_not_na, 'WS_BH_BOVENSTROOMS_L'], equal_nan=True)
# # GDF.loc[ either_not_na ].loc[ ~comparison, ['CODE', 'WS_BH_BOVENSTROOMS', 'WS_BH_BOVENSTROOMS_L', 'WS_BH_BOVENSTROOMS_']]
# #### - WS_BODEMBREEDTE_L_x
# compare_GDF_columns(GDF, 'WS_BODEMBREEDTE_L_x', rtol=0.001)
# GDF['rwid'] = GDF['WS_BODEMBREEDTE_L_x'].combine_first(GDF['WS_BODEMBREEDTE_L_y'])
# GDF.shape
# #### - X Y start end
# compare_GDF_columns(GDF, 'Xstart_x', atol=0.001, hide_y_na=True)
# compare_GDF_columns(GDF, 'Xend_x', atol=0.001, hide_y_na=True)
# compare_GDF_columns(GDF, 'Ystart_x', atol=0.001, hide_y_na=True)
# compare_GDF_columns(GDF, 'Yend_x', atol=0.001, hide_y_na=True)








# - **OVK00958** doesn't matter much, but _y is more accurate.
# - **OVK01666** "
# - **OVK02257** is split in two segments in _x (detail) while it's 1 feature in _y (1ry). It's safer to keep the more accurate _y version.
# - **OVK02272** is also more precise in _y. We'll keep the shape from _y as well.

# We'll prioritize _y values where they exist.
# for col in ['Xstart', 'Ystart', 'Xend', 'Yend']:
#     GDF[col] = GDF[f'{col}_y'].combine_first(GDF[f'{col}_x'])
# Same goes for the geometry
# GDF['geometry'] = GDF['geometry_y'].combine_first(GDF['geometry_x'])
# 'OBJECTID' in GDF_detail_copy.columns
# GDF['OBJECTID'].value_counts()
# GDF_ = GDF.copy()
# GDF = GDF_[ ['CODE', 'Length', 'rwid', 'Elv_UStr', 'Elv_DStr', 'Xstart', 'Xend', 'Ystart', 'Yend', 'geometry'] ].sort_values('CODE').reset_index(drop=True)
# GDF['OBJECTID'] = GDF.index + 1


# #### Limit X, Y to model area
# # A = U.GDF_clip_Mdl_Aa(A_all, Pa_INI=d_Pa['INI'])

In [ ]:
# # Debug: Check a specific non-split geometry
# test_code = non_split_codes[0]
# g3 = A3[A3['CODE'] == test_code].iloc[0]['geometry']
# g4 = A4[A4['CODE'] == test_code].iloc[0]['geometry']

# print(f"Code: {test_code}")
# print(f"A3 geometry type: {g3.geom_type}")
# print(f"A4 geometry type: {g4.geom_type}")

# if g3.geom_type == 'LineString':
#     print(f"A3 start: {g3.Coos[0]}")
#     print(f"A3 end: {g3.Coos[-1]}")
# else:  # MultiLineString
#     print(f"A3 start (first geom): {g3.geoms[0].Coos[0]}")
#     print(f"A3 end (last geom): {g3.geoms[-1].Coos[-1]}")
    
# if g4.geom_type == 'LineString':
#     print(f"A4 start: {g4.Coos[0]}")
#     print(f"A4 end: {g4.Coos[-1]}")
# else:  # MultiLineString
#     print(f"A4 start (first geom): {g4.geoms[0].Coos[0]}")
#     print(f"A4 end (last geom): {g4.geoms[-1].Coos[-1]}")
    
# print(f"\nGeometries equal: {g3.equals(g4)}")
# print(f"Geometries exact equal: {g3.equals_exact(g4, tolerance=0)}")

# # Check if shapely.set_precision is changing the geometry type
# import shapely
# g3_snapped = shapely.set_precision(g3, 0.001)
# print(f"\nA3 after set_precision type: {g3_snapped.geom_type}")
# print(f"A3 original == snapped: {g3.equals_exact(g3_snapped, tolerance=0)}")
